In [2]:
import json
import pickle
import numpy
import pandas
from pandas import DataFrame
pandas.set_option('display.max_rows', 100)
epLat = 0.012160099999998054
epLng = 0.014610749999992125
epTime = 255


In [3]:
with open('trajTrain.pickle','rb') as f:
    trajTrainDict = pickle.load(f)
    trajTrain = trajTrainDict['trajTrain']
    trajTrainOri = trajTrainDict['trajTrainOri']
    mappingTrain = trajTrainDict['mappingTrain']
    del trajTrainDict

In [3]:
with open('trajTest.pickle','rb') as f:
    trajTestDict = pickle.load(f)
    trajTest = trajTestDict['trajTest']
    trajTestOri = trajTestDict['trajTestOri']
    mappingTest = trajTestDict['mappingTest']
    del trajTestDict

In [4]:
from math import cos
from math import sin
from math import radians
from math import asin
import numpy

def getDistance(x,y):
	Long1,Lat1 = x
	Long2,Lat2 = y
	deltaX = cos(radians(Lat2))*cos(radians(Long2)) \
			 - cos(radians(Lat1)) * cos(radians(Long1))
	deltaY = cos(radians(Lat2))*sin(radians(Long2)) \
			 - cos(radians(Lat1)) * sin(radians(Long1))
	deltaZ = sin(radians(Lat2)) - sin(radians(Lat1))
	C = numpy.linalg.norm(numpy.array([deltaX, deltaY, deltaZ]))
	deltaSigma = 2 * asin(C/2)
	d = round( 6371 * 1000 * deltaSigma )
	return d

In [5]:
LongColNum = 3
LatColNum = 4
import math

def getDegree(p1,p2):
            a = getDistance((p2[LongColNum],p1[LatColNum]),
                            (p1[LongColNum],p1[LatColNum])
                            )
            if (p2[LongColNum] - p1[LongColNum]) < 0:
                a = - a
            
            b = getDistance((p1[LongColNum],p2[LatColNum]),
                            (p1[LongColNum],p1[LatColNum]))
            if (p2[LatColNum] - p1[LatColNum]) < 0:
                b = - b
            
            result = round(math.degrees(math.atan2(b,a)))
            return result

In [6]:
#trajTrain
def getLineDf(trajTrain):
  trajNoArr = numpy.unique(trajTrain[5])
  trajTrain2 = []
  for aTrajNo in trajNoArr:
      OneTrajTrain = trajTrain.loc[trajTrain[5]==aTrajNo,]
      if OneTrajTrain.size <= 0 or OneTrajTrain.shape[0] <= 1:
          continue
      nCubes = OneTrajTrain.shape[0]
      j=0
      for indCube in range(nCubes-1):
          oneCube = OneTrajTrain.iloc[indCube,]
          secondCube = OneTrajTrain.iloc[indCube+1,]
          listNewLine = [    
              None,
              j,
              None,
              None,
              None,
              oneCube[5],
              oneCube[0],
              secondCube[0],
              getDegree(oneCube,secondCube),
              oneCube[2], 
              secondCube[2],      
              oneCube[3],
              oneCube[4],
              secondCube[3],
              secondCube[4] ]
          trajTrain2.append(listNewLine)
          j += 1
  trajTrain2 = DataFrame(trajTrain2,columns=list(range(15)))
  trajTrain2[5] = trajTrain2[5].astype('int')
  trajTrain2[6] = trajTrain2[6].astype('int')
  trajTrain2[7] = trajTrain2[7].astype('int')
  trajTrain2[8] = trajTrain2[8].astype('int')
  trajTrain2[9] = trajTrain2[9].astype('int')
  trajTrain2[10] = trajTrain2[10].astype('int')
  return trajTrain2



In [7]:
def ignoreBeginOrLastOnePointCube(traj,trajOri,mapping):
    hist = numpy.bincount(mapping[4])
    hist2 = numpy.bincount(hist)
    ratioOfMostGap = 0.2
    lessGap = numpy.where((hist2.cumsum() / hist2.sum()) <= ratioOfMostGap)[0][-1] 
    #get lessGap
    n = hist.__len__()
    indexTop = 0
    indexTop2 = 0
    for i in range(n):
        if hist[i] <= lessGap :
            indexTop += 1
            indexTop2 += 1
        else:
            break
    indexEnd = traj.shape[0] - 1
    indexEnd2 = trajOri.shape[0] - 1
    for i in range(n-1,0,-1):
        if hist[i] <= lessGap :
            indexEnd -= 1
            indexEnd2 -= 1
        else:
            break
    return (traj.iloc[indexTop:(indexEnd+1),:].copy(),trajOri.iloc[indexTop2:(indexEnd2+1),:].copy())
        

In [8]:
ratioOfMostGap = 0.7

def getLineDf2(trajTrain,trajTrainOri,trajMapping):
  trajNoArr = numpy.unique(trajTrain[5])
  trajTrain2 = []
  for aTrajNo in trajNoArr:
      OneTrajTrain = trajTrain.loc[trajTrain[5]==aTrajNo,]
      OneTrajOri = trajTrainOri.loc[trajTrainOri[5]==aTrajNo,]
      OneTrajMapping = trajMapping.loc[trajMapping[2] == aTrajNo,]
      OneTrajTrain, OneTrajOri = ignoreBeginOrLastOnePointCube(OneTrajTrain, OneTrajOri, OneTrajMapping)
      
      
      timeDiff = targetTrajOri[2].iloc[1:].to_numpy() - targetTrajOri[2].iloc[:-1].to_numpy()
      timeDiff = timeDiff.astype('int')
      timeDiff = numpy.sort(timeDiff)
      iarr = math.ceil(timeDiff.size * ratioOfMostGap)
      timeDiffTop = timeDiff[iarr] * 2      
      
      if OneTrajTrain.size <= 0 or OneTrajTrain.shape[0] <= 1:
          continue
      nCubes = OneTrajTrain.shape[0]
      j=0
      biasTime = 0
      for indCube in range(nCubes-1):
          oneCube = OneTrajTrain.iloc[indCube,]
          secondCube = OneTrajTrain.iloc[indCube+1,]
          cubeTimeDiff = secondCube[9] - oneCube[10]
          if cubeTimeDiff > timeDiffTop:
                biasTime += secondCube[2] - oneCube[2]
                continue
          listNewLine = [    
              None,
              j,
              None,
              None,
              None,
              oneCube[5],
              oneCube[0],
              secondCube[0],
              getDegree(oneCube,secondCube),
              oneCube[2]-biasTime, 
              secondCube[2]-biasTime,      
              oneCube[3],
              oneCube[4],
              secondCube[3],
              secondCube[4] ]
          trajTrain2.append(listNewLine)
          j += 1
  trajTrain2 = DataFrame(trajTrain2,columns=list(range(15)))
  trajTrain2[5] = trajTrain2[5].astype('int')
  trajTrain2[6] = trajTrain2[6].astype('int')
  trajTrain2[7] = trajTrain2[7].astype('int')
  trajTrain2[8] = trajTrain2[8].astype('int')
  trajTrain2[9] = trajTrain2[9].astype('int')
  trajTrain2[10] = trajTrain2[10].astype('int')
  return trajTrain2

In [204]:
trajTest3 = getLineDf2(trajTest,trajTestOri,mappingTest)

In [30]:
trajTrain2 = getLineDf(trajTrain)
trajTest2 = getLineDf(trajTest)

In [108]:
trajTest3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,None,0,None,None,None,1,8,7,36,887,1017,104.083924,30.617963,104.098005,30.626660
1,None,1,None,None,None,1,7,7,46,1017,1162,104.098005,30.626660,104.111630,30.638730
2,None,2,None,None,None,1,7,5,-6,1162,1312,104.111630,30.638730,104.127317,30.637199
3,None,3,None,None,None,1,5,8,-20,1312,1510,104.127317,30.637199,104.139791,30.633233
4,None,0,None,None,None,2,9,9,-22,2554,2732,104.130673,30.636567,104.141061,30.632876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743,None,3,None,None,None,1399,6,6,-32,2368,2642,104.051068,30.678161,104.065704,30.670345
6744,None,4,None,None,None,1399,6,3,-68,2642,2884,104.065704,30.670345,104.071008,30.658982
6745,None,0,None,None,None,1400,5,5,115,1282,1488,104.033414,30.693410,104.026048,30.706766
6746,None,1,None,None,None,1400,5,7,73,1488,1745,104.026048,30.706766,104.030536,30.719723


In [55]:
with open('trajTrain2.pickle','wb') as f:
	pickle.dump(trajTrain2, f)
with open('trajTest2.pickle','wb') as f:
	pickle.dump(trajTest2, f)

In [26]:

trajTrain2.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,None,0,None,None,None,1,4,3,-129,1523,1788,104.111844,30.642299,104.103978,30.634074
1,None,1,None,None,None,1,3,2,-118,1788,2083,104.103978,30.634074,104.099186,30.626159
2,None,2,None,None,None,1,2,4,-149,2083,2348,104.099186,30.626159,104.089805,30.621353
3,None,3,None,None,None,1,4,4,173,2348,2559,104.089805,30.621353,104.076066,30.622818
4,None,4,None,None,None,1,4,5,114,2559,2809,104.076066,30.622818,104.074889,30.625046
5,None,0,None,None,None,2,5,4,-72,1324,1552,104.104765,30.685733,104.110146,30.671058
6,None,1,None,None,None,2,4,4,-75,1552,1774,104.110146,30.671058,104.114124,30.657857
7,None,2,None,None,None,2,4,5,-69,1774,2032,104.114124,30.657857,104.118165,30.648976
8,None,3,None,None,None,2,5,4,-132,2032,2267,104.118165,30.648976,104.112330,30.643386
9,None,4,None,None,None,2,4,2,-134,2267,2413,104.112330,30.643386,104.106731,30.638470


In [33]:
  trajTest2[5] = trajTest2[5].astype('int')
  trajTest2[6] = trajTest2[6].astype('int')
  trajTest2[7] = trajTest2[7].astype('int')
  trajTest2[8] = trajTest2[8].astype('int')
  trajTest2[9] = trajTest2[9].astype('int')
  trajTest2[10] = trajTest2[10].astype('int')
trajTest2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,None,0,None,None,None,1,8,7,36,887,1017,104.083924,30.617963,104.098005,30.626660
1,None,1,None,None,None,1,7,7,46,1017,1162,104.098005,30.626660,104.111630,30.638730
2,None,2,None,None,None,1,7,5,-6,1162,1312,104.111630,30.638730,104.127317,30.637199
3,None,3,None,None,None,1,5,8,-20,1312,1510,104.127317,30.637199,104.139791,30.633233
4,None,0,None,None,None,2,9,9,-22,2554,2732,104.130673,30.636567,104.141061,30.632876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8196,None,4,None,None,None,1399,6,3,-68,2642,2884,104.065704,30.670345,104.071008,30.658982
8197,None,0,None,None,None,1400,5,5,115,1282,1488,104.033414,30.693410,104.026048,30.706766
8198,None,1,None,None,None,1400,5,7,73,1488,1745,104.026048,30.706766,104.030536,30.719723
8199,None,2,None,None,None,1400,7,2,109,1745,2043,104.030536,30.719723,104.026707,30.729176


In [9]:
def getOverlapOneLine(trajList1,trajList2,dtime,distX,distY,angDiffConst,pointAmountRatio = 0):
    maxTime = trajList1[9] + dtime
    minTime = trajList1[9] - dtime
    #trajList3 = trajList2.loc[(trajList2[9] > minTime) & (trajList2[9] < maxTime),]
    #trajList4 = trajList3.loc[abs(trajList3[9] - trajList1[9]) < dtime,]
    trajList4 = trajList2
    trajList4 = trajList4.loc[abs(trajList4[11] - trajList1[11]) < distX,]
    trajList4 = trajList4.loc[abs(trajList4[12] - trajList1[12]) < distY,]
    trajList4 = trajList4.loc[abs(trajList4[13] - trajList1[13]) < distX,]
    trajList4 = trajList4.loc[abs(trajList4[14] - trajList1[14]) < distY,]
    
    trajList4 = trajList4.loc[angleDiff(trajList4[8],trajList1[8]) < angDiffConst,]
    
    while pointAmountRatio > 0:
        trajList5 = trajList4.loc[abs(trajList4[6] - trajList1[6]) < (trajList1[6] * (1-pointAmountRatio) ),]
        if trajList5.size > 0:
            trajList4 = trajList5
            break
        pointAmountRatio -= 0.1
    while pointAmountRatio > 0:
        trajList5 = trajList4.loc[abs(trajList4[7] - trajList1[7]) < (trajList1[7] * (1-pointAmountRatio) ),]
        if trajList5.size > 0:
            trajList4 = trajList5
            break
        pointAmountRatio -= 0.1
        
    if trajList4.size <= 0:
        return None
    n1 = trajList4.shape[0]
    area1 = DataFrame(numpy.zeros((n1,3), dtype=int))
    area1[0] = trajList4[5].values  #其它軌
    area1[1] = trajList4[1].values  #其它軌第幾cube
    area1[2] = trajList4.index.values  #其它軌此cube在總名單上第幾
    return area1

In [10]:
def angleDiff(a ,b):
	result = abs(a - b)
	result = result.values
	if len(a) > 1:
		index = result > 180
		result[index] = 360 - result[index]
	else:
		if result > 180:
			result = 360 - result
	return result

In [11]:
def getMaxGroup(trajList):
	aList = trajList[10] - trajList[9]
	arr = numpy.array(aList)
	result = numpy.zeros(arr.size)
	n = arr.size
	ratio = n // 2
	for i in range(n):
		x1 = abs(arr - arr[i])
		x2 = numpy.sort(x1)
		result[i] = sum(x2[:ratio])
	minLoc = numpy.where(result == numpy.amin(result))
	if minLoc[0][0] >= 0:
		return arr[minLoc[0][0]]
	else:
		raise Exception('ERROR: getMaxGroup: minLoc[0][0] = %i < 0' % minLoc[0][0])

In [12]:
#targetTrajNo = 60

def getTestResult4OneTarget(targetTrajNo,
                            trajTest2,
                            trajTrain2,
                            epLng,epLat,
                           delta2,epLng2,epLat2,phi2,pointRatio2
                           ):
    targetTraj = trajTest2.loc[trajTest2[5] == targetTrajNo,].copy()
    targetTrajOri = trajTestOri.loc[trajTestOri[5] == targetTrajNo,].copy()
    #targetMapping = mappingTest.loc[mappingTest[2] == targetTrajNo,].copy()
    #targetNew,targetOriNew = ignoreBeginOrLastOnePointCube(targetTraj,targetTrajOri,targetMapping)
    targetNew = targetTraj.copy()
    targetNew2 = targetTraj.copy()
    targetOriNew = targetTrajOri.copy()
    realTime = targetNew.iloc[-1,10]-targetNew.iloc[0,9]
    totalTime = 0
    n = targetNew.shape[0]
    commonTime = 0
    tempExpResult = numpy.zeros((n,2),dtype='int')
    tempExpResult[:,0] = targetNew.iloc[:,10] - targetNew.iloc[:,9]
    listArea = []
    for i2 in range(n):
        if i2 > 0:
                targetNew.iloc[i2,9] = targetNew.iloc[i2-1, 9] + commonTime
        epLngTemp = epLng2
        epLatTemp = epLat2
        #for i3 in range(5):
        area = getOverlapOneLine(targetNew.iloc[i2,],
                                 trajTrain2,
                                 delta2,epLngTemp,epLatTemp,phi2,pointRatio2)
        #    if area is None:
        #        epLngTemp += epLng2
        ##        epLatTemp += epLat2
        #    else:
        #        break
        if area is None:
            print('None apear, i2=',i2)
            return None
        if area.shape[0] <= 1:
            print('area.shape[0] <= 1, area.shape[0]=',area.shape[0])
        listArea.append(area)
        commonTime = getMaxGroup(trajTrain2.loc[area[2],])
        tempExpResult[i2,1] = commonTime
        totalTime += commonTime
    return (totalTime,realTime,tempExpResult,listArea)

In [248]:
targetTrajNo = 13
totalTime,realTime,tempExpResult,listArea = getTestResult4OneTarget(targetTrajNo,
                            trajTest3,
                            trajTrain2,
                            epLng,epLat,
                            3600,epLng/4,epLat/4,10,0.5)

In [249]:
print(totalTime)
print(realTime)

640
500


In [247]:
tempExpResult

In [194]:
trajTrain2.loc[listArea[1][2],]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
34598,None,6,None,None,None,5836,5,2,162,3311,3452,104.075542,30.635488,104.068088,30.637547


In [197]:
targetTrajOri = trajTestOri.loc[trajTestOri[5]==14,]
timeDiff = targetTrajOri[2].iloc[1:].to_numpy() - targetTrajOri[2].iloc[:-1].to_numpy()
timeDiff = timeDiff.astype('int')
timeDiff = numpy.sort(timeDiff)
timeDiff


array([  19,   20,   30,   30,   30,   30,   30,   40,   50,   50,  179,
        191,  373, 1080])

In [202]:
iarr = math.ceil(timeDiff.size * 0.6)
timeDiff[iarr]
#timeDiffTop = timeDiff[iarr] * 2
#timeDiffTop

50

In [213]:
trajNoList = numpy.unique(trajTest3[5])

expResult = numpy.zeros((trajNoList.max()+1,5),dtype='int')
#expResult[:,2] = 0
expResult[:,4] = list(range(trajNoList.max()+1))
#
#expTotalResult = 0



In [214]:
#[0:100]
for targetTrajNo in trajNoList:
    print(targetTrajNo, end=' ')
    result = getTestResult4OneTarget(targetTrajNo,
                            trajTest3,
                            trajTrain2,
                            epLng,epLat,
                            3600,epLng/4,epLat/4,10,0.5)
    if result is None:
        totalTime = 0
        realTime = 0
        tempExpResult = None
    else:
        expResult[targetTrajNo,0] = 1
        totalTime,realTime,tempExpResult,listArea = result
    expResult[targetTrajNo,1] = totalTime
    expResult[targetTrajNo,3] = realTime

1 2 3 4 

/home/oscar/anaconda3/envs/hypercube/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


None apear, i2= 2
5 None apear, i2= 2
6 None apear, i2= 3
7 8 9 10 11 12 13 14 15 16 17 18 19 20 None apear, i2= 5
21 22 23 24 25 26 None apear, i2= 0
27 None apear, i2= 5
28 None apear, i2= 5
29 None apear, i2= 0
30 31 32 None apear, i2= 4
33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 None apear, i2= 0
48 None apear, i2= 3
49 50 51 52 53 54 55 56 57 None apear, i2= 1
58 59 60 None apear, i2= 8
61 62 None apear, i2= 0
63 None apear, i2= 0
64 None apear, i2= 0
65 66 67 68 69 None apear, i2= 2
70 71 None apear, i2= 1
72 73 74 75 76 77 78 79 80 81 None apear, i2= 1
82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 None apear, i2= 0
98 99 100 None apear, i2= 1
101 102 103 None apear, i2= 2
104 105 106 107 108 109 None apear, i2= 1
110 111 None apear, i2= 0
112 113 114 115 116 117 None apear, i2= 0
118 119 120 121 122 None apear, i2= 8
123 None apear, i2= 2
124 125 None apear, i2= 0
126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 None apear, i2= 1
141 None apear, i2= 0
142 None ape

1011 None apear, i2= 0
1012 None apear, i2= 0
1013 1014 1015 None apear, i2= 1
1016 None apear, i2= 0
1017 1018 1019 1020 1021 1022 1023 None apear, i2= 4
1024 None apear, i2= 6
1025 1026 1027 1028 None apear, i2= 1
1029 None apear, i2= 0
1030 1031 None apear, i2= 3
1032 None apear, i2= 2
1033 1034 1035 1036 1037 1038 None apear, i2= 0
1039 None apear, i2= 2
1040 None apear, i2= 1
1041 None apear, i2= 0
1042 1043 1044 None apear, i2= 6
1045 1046 None apear, i2= 2
1047 1048 1049 None apear, i2= 6
1050 1051 1052 None apear, i2= 0
1053 1054 1055 1056 1057 1058 1059 1060 1061 1062 1063 1064 1065 1066 1067 1068 None apear, i2= 0
1069 None apear, i2= 0
1070 1071 1072 1073 1074 1075 None apear, i2= 0
1076 None apear, i2= 7
1077 None apear, i2= 12
1078 1079 1080 1081 1082 None apear, i2= 2
1083 1084 1085 None apear, i2= 4
1086 1087 1088 1089 1090 1091 1092 None apear, i2= 0
1093 1094 1095 1096 1097 None apear, i2= 3
1098 1099 None apear, i2= 3
1100 1101 1102 1103 1104 1105 1106 1107 1108 None 

In [133]:
#test = expResult[numpy.where(expResult[:,2] == 1)]
tempExpResult

array([[112, 239],
       [107, 264],
       [103, 242],
       [115, 255],
       [201, 256],
       [194, 266]])

In [183]:
trajTest3.loc[trajTest3[5] == 14,]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
72,None,0,None,None,None,14,6,3,168,2222,2441,104.086630,30.642662,104.076076,30.644613
73,None,1,None,None,None,14,2,3,168,2441,3016,104.073052,30.637443,104.069523,30.638068


In [185]:
trajTest2.loc[trajTest2[5] == 14,]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
84,None,0,None,None,None,14,6,3,168,2222,2441,104.086630,30.642662,104.076076,30.644613
85,None,1,None,None,None,14,3,2,-118,2441,2756,104.076076,30.644613,104.073299,30.640135
86,None,2,None,None,None,14,2,2,-95,2756,3694,104.073299,30.640135,104.073052,30.637443
87,None,3,None,None,None,14,2,3,168,3694,4269,104.073052,30.637443,104.069523,30.638068


In [218]:
expResult

array([[   0,    0,    0,    0,    0],
       [   1,  586,    0,  623,    1],
       [   1, 2642,    0, 2384,    2],
       ...,
       [   1,  743,    0,  674, 1398],
       [   1, 1299,    0, 1339, 1399],
       [   0,    0,    0,    0, 1400]])

In [239]:
expResult2 = expResult[numpy.where(expResult[:,0] == 1)]
deviation = numpy.absolute(expResult2[:,1] - expResult2[:,3])
eachAPE = deviation * 100 / expResult2[:,3] 
eachAPE = eachAPE.astype(int)
#eachAPE[[49,6,11,60]] = 0
eachAPE

array([ 5, 10, 37, ..., 19, 10,  2])

In [238]:
#expResult[ numpy.where( (expResult[:,1] == 0) &(expResult[:,3] == 0) ),0 ] = 0
#expResult[597][0] = 0
expResult2[706]
expResult[920][0] = 0

In [240]:
eachAPE.mean()

12.959662288930582

In [250]:
#eachAPE[eachAPE > 50]
numpy.where(eachAPE > 50)

(array([   3,  153,  158,  197,  253,  289,  393,  503,  625,  713,  717,
         755,  756,  775,  838,  902,  906,  945, 1057]),)

In [245]:
expResult2[36]

array([   1, 1252,    0, 1362,   46])

In [256]:
trajTest3.loc[[73,72],9]

73    2460
72    2926
Name: 9, dtype: int64

In [4]:
trajTrainOri

,0,1,2,3,4,5
0,0,0,1463.0,104.115353,30.643920,1
1,0,1,1503.0,104.113091,30.642129,1
2,0,2,1543.0,104.110404,30.643930,1
3,0,3,1583.0,104.108335,30.640667,1
4,0,4,1754.0,104.106304,30.637807,1
...,...,...,...,...,...,...
662318,0,24,2194.0,104.095948,30.675855,18000
662319,0,25,2224.0,104.097979,30.673126,18000
662320,0,26,2254.0,104.099605,30.672648,18000
662321,0,27,2284.0,104.103675,30.673848,18000


In [5]:
mappingTrain

,0,1,2,3,4
0,0,0,1,0,0
1,0,0,1,1,0
2,0,0,1,2,0
3,0,0,1,3,0
4,0,0,1,4,1
...,...,...,...,...,...
662318,0,0,18000,24,3
662319,0,0,18000,25,3
662320,0,0,18000,26,3
662321,0,0,18000,27,4
